In [ ]:
import numpy as np
import nltk
from nltk.corpus import reuters
import string
import re
from random import randint
import math
import argparse, sys
import time

W = np.loadtxt('data/W_NEG_15_DIM_300_EPOCHS_100_cont2.txt')

In [40]:
len(W)

300

In [41]:
len(W[0])

28354

In [4]:
def cosineDistance(x, y):
    s = np.dot(x,y)
    s = s/((np.dot(x,x))**0.5)
    s = s/((np.dot(y,y))**0.5)
    return s

def cosineDistanceString(s1, s2, W, word2id):
    x = W[word2id[s1]]
    y = W[word2id[s2]]
    return cosineDistance(x, y)

In [5]:
def readVocabulary(fileName):
    f1 = open(fileName, "r")
    vocab = {}
    for line in f1:
        v = line.strip().split('\t')
        vocab[v[0]] = int(v[1])
    f1.close()
    return vocab

def readWord2id(fileName):
    f1 = open(fileName, "r")
    word2id = {}
    for line in f1:
        v = line.strip().split('\t')
        word2id[v[0]] = int(v[1])
    f1.close()
    return word2id

def readId2word(fileName):
    f1 = open(fileName, "r")
    id2word = {}
    for line in f1:
        v = line.strip().split('\t')
        id2word[int(v[0])] = v[1]
    f1.close()
    return id2word

def readAll():
    vocab = readVocabulary('data/vocab.txt')
    word2id = readWord2id('data/word2id.txt')
    id2word = readId2word('data/id2word.txt')
    return vocab, word2id, id2word


In [6]:
vocab, word2id, id2word = readAll()


In [7]:
len(word2id)

28354

In [ ]:
W = W.T

In [12]:

embedding_king = W[word2id['queen']]

In [13]:
word_to_eval = 'far'
closest_word = ''
closest_word_dist = -9999
for i in range(len(W)):
    word = id2word[i]
    dist = cosineDistance(W[word2id[word_to_eval]], W[i])
    if(dist > closest_word_dist and i != word2id[word_to_eval] and i != word2id['craft']):
        closest_word_dist = dist
        closest_word = word
closest_word



'so'

In [20]:
cosineDistanceString('good', 'bad', W, word2id)

0.18763342851069334

In [21]:
cosineDistance(W[word2id['queen']],W[word2id['king']])

0.12729128717000557

In [82]:
C = np.loadtxt('data/C_NEG_15_DIM_300_EPOCHS_100_cont2.txt')

In [22]:
def findClosest(word_to_eval, W, word2id, id2word):
    closest_word = ''
    closest_word_dist = -9999
    for i in range(len(W)):
        word = id2word[i]
        dist = cosineDistance(W[word2id[word_to_eval]], W[i])
        if(dist > closest_word_dist and i != word2id[word_to_eval]):
            closest_word_dist = dist
            closest_word = word
    return closest_word


In [23]:
ans = findClosest('but', W, word2id, id2word)
ans

'schenker'

In [26]:
cosineDistanceString('but', 'schenker', W, word2id)

0.3456299886244699

In [27]:
f = open("SimLex-999.txt", "r")
scores = []
i = 0
for line in f:
    i+= 1
    if(i == 1):
        continue
    line = line.strip()
    tokens = line.split('\t')
    sample = {}
    sample['w1'] = tokens[0]
    sample['w2'] = tokens[1]
    sample['score'] = float(tokens[3])
    if sample['w1'] in vocab and sample['w2'] in vocab:
        scores.append(sample)

In [43]:
len(scores)

617

In [44]:
from scipy.stats import pearsonr
predicted_scores = []
simlex_score = []
for sample in scores:
    w1 = sample['w1']
    w2 = sample['w2']
    predicted_scores.append(cosineDistanceString(w1, w2, W, word2id))
    simlex_score.append(sample['score'])
corr, p_value = pearsonr(simlex_score, predicted_scores)
corr

0.043968636652993796

In [31]:
C = np.loadtxt('data/C_NEG_15_DIM_300_EPOCHS_100_cont2.txt')

In [35]:
avg = (W+C)/2